# Finding the difference between the S-learner and the T-learner

Here you can find the solution to the exercise in Chapter 4. We will use
the library `scikit learn` from Python that trains decision trees.
First, we set the parameters (we used the function `numpy.random.seed`
to make the example is reproducible).

In [1]:
from sklearn.tree import DecisionTreeRegressor
from numpy.random import normal, uniform, seed
from numpy import exp, mean
from pandas import DataFrame

seed(54321)

n = 20
max_depth = 5
b_z_x = 5
b_z_y = 5
b_x_y = 2
sd = 1

# 1. Generate the data

In [2]:
z = normal(size=n, scale=sd)
x = (uniform(size=n) < 1 / (1 + exp(-b_z_x * z))).astype(int)
y = b_z_y * z + b_x_y * x + normal(size=n, scale=sd)
df = DataFrame({"z": z, "x": x, "y": y})

Note that if we tried to see the difference of outcome y, as a naive
estimation of the impact of x into y, we would see that the result is
7.24, quite far from the actual value.

# 2. Calculate the difference between groups of outcome y

In [3]:
df.query("x==1").y.mean() - df.query("x==0").y.mean()

7.2363405261057805

Third, we use the S-learner. For that we train a decision tree, and
create two new datasets, one with variable x set 0 zero in all
observations and another with x=1 in all observations. Finally, the ATE
is estimated as the mean difference of the prediction of the trained
model in both new datasets.

# 3. S-learner

In [4]:
model = DecisionTreeRegressor(max_depth=max_depth)
X = df[["x", "z"]]
y = df["y"]
model.fit(X, y)


df_do_0 = df.copy()
df_do_0.x = 0
predictions_0 = model.predict(df_do_0[["x", "z"]])

df_do_1 = df.copy()
df_do_1.x = 1
predictions_1 = model.predict(df_do_1[["x", "z"]])

print("ATE")
print(mean(predictions_1 - predictions_0))
print(predictions_1 - predictions_0)

ATE
0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

You will get an ATE of 0. Moreover, the differences for each observation
(predictions_1 – predictions_0) are also 0. Notice that this happens for
a particular choice of seed. In general, if you change the seed, you
will see that not always you have an ATE of 0. However, a numerical
methods that from time to time gives an incorrect answer, zero, is not a
good numerical method.

Let’s see how the T-Learner works now. First, we partition the dataset
into the observations with x = 0 and with x = 1. Then we train a model
for each group, and use both models to predict for each observation,
what would be the expected outcome y when x is set to 0 and the same for
x equal 1.

# 4. T-Learner

In [5]:
df_0 = df.query("x==0")
df_1 = df.query("x==1")

model_0 = DecisionTreeRegressor(max_depth=max_depth)
X_0 = df_0[["x", "z"]]
y_0 = df_0["y"]
model_0.fit(X_0, y_0)

model_1 = DecisionTreeRegressor(max_depth=max_depth)
X_1 = df_1[["x", "z"]]
y_1 = df_1["y"]
model_1.fit(X_1, y_1)

predictions_0 = model_0.predict(df[["x", "z"]])
predictions_1 = model_1.predict(df[["x", "z"]])

print("ATE")
print(mean(predictions_1 - predictions_0))
print(predictions_1 - predictions_0)

ATE
4.278751389023496
[ 3.24930881  2.61665433  3.56572722  6.09852166 15.18519343  3.74718771
  2.01039955  3.06142779  7.02389793  3.33348004  5.85945538  2.01039955
  1.93609629  2.26595439  2.61665433  2.61665433  3.24930881  5.80432186
  2.61665433  6.70773005]

You can see that the ATE in this case is 4.28, much better (closer to
the actual value of 2) than the difference of means of outcomes in group
x = 0 and x = 1, that were 7.23.

Notice also that we have set the maximum depth to 5. In a realistic
situation, maximum depth would be chosen via hyperparameter tunning, so
splitting data into many groups, calculating cross-validations and
trying different sets of parameters.